# Review of Typical Neural Network Framework

In any supervised or semi-supervised setting, we are given a set of inputs and outputs $\{(x_i, y_i)\}^n_i$. The goal is learn a function $F: x \rightarrow y$ that maps each input $x_i$ to its corresponding output $y_i$. Typically, $f$ is modeled as a neural network $f_\theta$ where the parameters $\theta$ are learned through stochastic gradient descent with respect to a user-defined loss function $\mathcal{L}(x_i, y_i)$ (e.g. squared loss).  

# Neural Ordinary Differential Equations (Neural ODEs)
Chen et al. showed that instead of modeling $F: x \rightarrow y$ as a neural network, we can reformulate the problem in a continuous setting. Namely, let us call $h(t)$ the **hidden state** which depends on $t$ ($t$ can but does not have to represent time). The initial hidden state at some time $t_0$ will be $h(t_0)=x$. The final hidden state at some time $t_1$ after $t_0$ will be $h(t_1) = y$. 

The derivative of $h(t)$ with respect to $t$ is modeled as a neural network $f$ which receives $h(t)$ and $t$ as input and is parameterized by parameters $\theta$. Notice that the values of the initial time $t_0$ and final time $t_1$ are not well-defined. Like the parameters of the neural network $\theta$, $t_0$ and $t_1$ are free parameters and are likewise optimized through gradient descent with respect to the loss $\mathcal{L}$.

In summary, we are learning a function $f(t, h(t), \theta)$ such that

$$f(t, h(t), \theta) = \frac{d h(t)}{d t}$$
$$h(t_0) = x$$
$$h(t_1) = y$$

Our prediction $\hat{y}$ is thus

$$\widehat{y} = h(t_1) = \text{ODESolve}(h(t_0), f, t_0, t_1, \theta) \big)$$

where $ODESolve$ is any numerical ODE solver (e.g. Runge-Kutta) which solves for the hidden state at $t_1$, $h(t_1)$. To optimize, the free parameters $t_0$, $t_1$, and $\theta$, the adjoint sensitivity method as described by Chen et al. is first used to find the gradients $\frac{d \mathcal{L}}{d h(t)}$ and $\frac{d \mathcal{L}}{d \theta}$ where $\mathcal{L}$ is a scalar loss defined as 

$$
L(h(t_1)) = L \Big( \int_{t_0}^{t_1} f(h(t), t, \theta)dt \Big) = L \big( \text{ODESolve}(h(t_0), f, t_0, t_1, \theta) \big)
$$

The free parameters are then optimized using gradient descent. 

## Adjoint Sensitivity Method

As alluded to previously, the **adjoint method** is a way to perfrom the backward pass for neural ODEs and compute the gradients with respect the parameters. Assume as before that $h(t_1)$ is the output of our model but this time $h(t_0)$ is just our input. The free parameters in this case are

* $t_0$: the initial time
* $t_1$: the final time
* $\theta$: parameters of $f$

We first define the **adjoint** $a(t)$ as the gradient of the loss $\mathcal{L}$ with respect to the state $h(t_1)$.

$$
a(t) = -\frac{\partial L}{\partial h(t)}
$$

Computing the gradient $\frac{\partial L}{t_1}$ is just

$$
\frac{\partial L}{\partial t_1} = - a(t) \frac{\partial f(h(t), t, \theta)}{\partial t}
$$

The computation for the other gradients is not as trivial. First, the derivative of the adjoint with respect to time is

$$
\frac{d a(t)}{d t} = -a(t) \frac{\partial f(h(t), t, \theta)}{\partial h}
$$

With a numerical solver, we can then solve this differential equation to obtain $\frac{\partial L}{\partial h(t_0)}$. This gradient is particularly useful for latent neural ODE models where the input is first transformed into latent space with an encoder neural network.

$$
\frac{\partial L}{\partial h(t_0)} = \int_{t_1}^{t_0} a(t) \frac{\partial f(h(t), t, \theta)}{\partial h} dt
$$

Note that the initial value of this differential equation $\frac{\partial L}{\partial h(t_1)}$ can be directly computed with normal backpropagation. We can likewise solve a two more backwards ODE to find the gradients for $\theta$ nad 

$$
\frac{\partial L}{\partial \theta} = \int_{t_1}^{t_0} a(t) \frac{\partial f(h(t), t, \theta)}{\partial \theta} dt
$$

$$
\frac{\partial L}{\partial t_0} = \int_{t_1}^{t_0} a(t) \frac{\partial f(h(t), t, \theta)}{\partial t} dt
$$

## Augmented Adjoint

We can solve all the backwards differntial equations at once by solving an augmented differential equation. First, we view the dynamics of $\theta$ and $t$ with respect to time as constant

$$\frac{d \theta}{dt} = 0$$
$$\frac{dt}{dt} = 1$$

We define the augmented dynamics $f_{aug}$ as 

$$
\frac{d}{dt} \begin{bmatrix} h \\ \theta \\ t \end{bmatrix} (t) = f_{\text{aug}}([h, \theta, t]) := \begin{bmatrix} f([h, \theta, t ]) \\ 0 \\ 1 \end{bmatrix} \tag{6}
$$

The adjoint of the augmented dynamics $a_{\text{aug}}$ is thus

$$
a_{\text{aug}} := \begin{bmatrix} a \\ a_{\theta} \\ a_t \end{bmatrix}, a_{\theta}(t) := \frac{\partial L}{\partial \theta(t)}, a_t(t) := \frac{\partial L}{\partial t(t)} \tag{7}
$$

The gradient of $f_{aug}$ is

$$
\frac{\partial f_{\text{aug}}}{\partial [h, \theta, t]} = \begin{bmatrix} 
\frac{\partial f}{\partial h} & \frac{\partial f}{\partial \theta} & \frac{\partial f}{\partial t} \\
0 & 0 & 0 \\
0 & 0 & 0
\end{bmatrix} \tag{8}
$$

The dynamics of the augmented adjoint state is

$$
\frac{d a_{\text{aug}}}{dt} = - \begin{bmatrix} a\frac{\partial f}{\partial z} & a\frac{\partial f}{\partial \theta} & a\frac{\partial f}{\partial t}\end{bmatrix} \tag{9}
$$

Using the algorithm below we compute can compute the gradients to optimize the free parameters

![aug_ode](./media/pseudocode.png)


## Advantages of Neural ODEs
There are several advantage of Neural ODEs over normal neural networks.

**First**, Neural ODEs ensure that **the learned mapping $F: x \rightarrow y$ is smooth**.

**Second**, the Neural ODE formulation is **well-suited for time series data**. For instance, in the framework given above, the only hidden states that had any meaning for our problem were $h(t_0$ which was our input $x$ and $h(t_1)$ which was our output $y$. All other hidden states in between $t_0$ and $t_1$ did carry any meaning for our problem. Imagine now that we are dealing with time-series data $x_0, x_1, x_2,...x_n)$ and our goal is to either interpolate in between two given data points or predict new data points. Now each hidden state has meaning $h(t) = x_t$.

**Third**, we can **take advantage of the rich differential equations theory and numerical solvers** developed over the past 200 years

# Limitations of Neural ODEs

One issue is that normal ODEs struggle with modeling functions that sudden jumps or spikes. Some dynamical systems found in nature may also be noisy (i.e. molecular dynamics, EEG, ...). Stochastic differential equations insert an additive noise term to the ordinary differential equation to model the noise in a system. Neural ODEs can be extended to Neural SDEs by modeling the noise term as a neural network. Neural ODEs are also usually more computationally expensive than normal neural networks because of having to use a solver.

# Neural ODE Demo
In the demo below, we will attempt to use a Neural ODE to fit time-series data from a molecular simulation of alanine dipeptide.